**Goal**: To Build an MCP server with gradio called Minimalistic Postcard that creates a tiny atmospheric postcard — built from weather, time, mood, pulse, and poetry.

**Install and Import all necessary packages**

In [ ]:
import requests
import os
import json
import openai
from openai import OpenAI
from pathlib import Path
import gradio as gr

**Set openai api key**

In [ ]:
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

**Define a tool to get the weather details of a place**

In [ ]:
def get_weather_by_place(place: str) -> dict:
    """Fetches current weather for a given place name using Open-Meteo APIs.

    This function first converts the place name to coordinates using the
    Open-Meteo Geocoding API, then retrieves current weather conditions
    (temperature, weather code, and cloud cover) from the Open-Meteo Weather API.

    Args:
        place: The name of the location (e.g., "New York", "Tokyo", "London").

    Returns:
        Dictionary with status and weather information.
        Success: {
            "status": "success",
            "place": "New York",
            "latitude": 40.71,
            "longitude": -74.01,
            "temperature_c": 21.3,
            "weather_code": 3,
            "cloud_cover": 78,
            "condition": "cloudy",
            "glyph": "▤",
        }
        Error: {
            "status": "error",
            "error_message": "Place not found or failed to fetch weather"
        }
    """

    # Step 1: Convert place name → coordinates
    geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={place}"
    try:
        geo_response = requests.get(geo_url, timeout=10)
        geo_response.raise_for_status()
        geo_data = geo_response.json()

        results = geo_data.get("results")
        if not results:
            return {"status": "error", "error_message": f"Place '{place}' not found"}

        location = results[0]
        latitude = location["latitude"]
        longitude = location["longitude"]
        resolved_name = location["name"]

    except Exception as e:
        return {"status": "error", "error_message": f"Geocoding failed: {str(e)}"}

    # Step 2: Fetch weather data for coordinates
    weather_url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={latitude}&longitude={longitude}"
        f"&current=temperature_2m,weathercode,cloudcover"
    )

    try:
        weather_response = requests.get(weather_url, timeout=10)
        weather_response.raise_for_status()
        weather_data = weather_response.json()

        current = weather_data.get("current", {})
        if not current:
            return {"status": "error", "error_message": "No current weather data found"}

        temperature = current.get("temperature_2m")
        weather_code = current.get("weathercode")
        cloud_cover = current.get("cloudcover")

        # Weather code → human description
        code_map = {
            0: "clear",
            1: "mainly clear",
            2: "partly cloudy",
            3: "cloudy",
            45: "foggy",
            48: "freezing fog",
            51: "light drizzle",
            61: "rainy",
            71: "snowy",
            95: "thunderstorm",
        }
        condition = code_map.get(weather_code, "unknown")

        # ---- NEW MINIMAL GLYPH (ONLY ADDITION) ----
        glyph_map = {
              "clear":            "─☀︎",
              "mainly clear":     "▁☼",
              "partly cloudy":    "⋅☁︎",
              "cloudy":           "▤☁︎",
              "foggy":            "≡≡",
              "freezing fog":     "≡*",
              "light drizzle":    "∿·",
              "rainy":            "∼∼",
              "snowy":            "❄⋅",
              "thunderstorm":     "⚡∼",
              "unknown":          "·",
              }
        glyph = glyph_map.get(condition, "·")

        # --------------------------------------------

        return {
            "status": "success",
            "place": resolved_name,
            "latitude": latitude,
            "longitude": longitude,
            "temperature_c": temperature,
            "weather_code": weather_code,
            "cloud_cover": cloud_cover,
            "condition": condition,
            "glyph": glyph,
        }

    except Exception as e:
        return {"status": "error", "error_message": f"Weather fetch failed: {str(e)}"}

**Create a tool to get the time of a place**

In [ ]:
def get_time_by_place(place: str) -> dict:
    """Fetches the current local time for a given place name using Open-Meteo.

    This function converts the place name to coordinates via the
    Geocoding API, then retrieves the local time and timezone from the
    Weather Forecast API (timezone=auto).

    Args:
        place: Name of the location (e.g., "Paris", "Tokyo").

    Returns:
        Success:
        {
            "status": "success",
            "place": "Paris",
            "latitude": 48.8566,
            "longitude": 2.3522,
            "timezone": "Europe/Paris",
            "local_datetime": "2025-11-22T14:43",
            "time_of_day": "afternoon",
            "time_mood": "bright and energetic"
        }

        Error:
        { "status": "error", "error_message": "..." }
    """
    # Step 1: Get coordinates
    geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={place}"
    try:
        geo_response = requests.get(geo_url, timeout=10)
        geo_response.raise_for_status()
        geo_data = geo_response.json()

        results = geo_data.get("results")
        if not results:
            return {"status": "error", "error_message": f"Place '{place}' not found"}

        location = results[0]
        latitude = location["latitude"]
        longitude = location["longitude"]
        resolved_name = location["name"]

    except Exception as e:
        return {"status": "error", "error_message": f"Geocoding failed: {str(e)}"}

    # Step 2: Fetch time using forecast API with timezone=auto
    time_url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={latitude}&longitude={longitude}"
        f"&timezone=auto&current=temperature_2m"
    )

    try:
        time_response = requests.get(time_url, timeout=10)
        time_response.raise_for_status()
        time_data = time_response.json()

        timezone = time_data.get("timezone")
        current = time_data.get("current", {})
        local_time = current.get("time")

        if not timezone or not local_time:
            return {"status": "error", "error_message": "Failed to retrieve local time"}

        # Determine time-of-day category
        hour = int(local_time[11:13])

        if 5 <= hour < 8:
            time_of_day = "sunrise"
            mood = "soft and calm"
        elif 8 <= hour < 12:
            time_of_day = "morning"
            mood = "fresh and lively"
        elif 12 <= hour < 17:
            time_of_day = "afternoon"
            mood = "bright and energetic"
        elif 17 <= hour < 20:
            time_of_day = "golden hour"
            mood = "warm and gentle"
        elif 20 <= hour < 22:
            time_of_day = "evening"
            mood = "quiet and reflective"
        else:
            time_of_day = "night"
            mood = "calm and dreamy"

        # ---- NEW EXPRESSIVE SUN/MOON GLYPHS ----
        glyph_map = {
            "sunrise": "↗︎☼",
            "morning": "☼",
            "afternoon": "☀︎",
            "golden hour": "☼⌇",
            "evening": "☾⌇",
            "night": "☾",
        }
        time_glyph = glyph_map.get(time_of_day, "·")
        # -----------------------------------------

        return {
            "status": "success",
            "place": resolved_name,
            "latitude": latitude,
            "longitude": longitude,
            "timezone": timezone,
            "local_datetime": local_time,
            "time_of_day": time_of_day,
            "time_mood": mood,
            "glyph": time_glyph,
        }

    except Exception as e:
        return {"status": "error", "error_message": f"Time fetch failed: {str(e)}"}

**Create a tool that creates a short poetic quote based on location, weather, time, and user-provided context (such as an activity or mood)**

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def generate_place_quote(
    place: str,
    weather: dict,
    time: dict,
    user_context: str = ""
) -> dict:
    """
    Generates a short poetic quote based on location, weather, time, and optional
    user-provided context (such as an activity or mood).

    Args:
        place: Name of the place (e.g., "Paris").
        weather: Weather dictionary from get_weather_by_place().
        time: Time dictionary from get_time_by_place().
        user_context: Optional string describing what the user is doing or wants included
                      (e.g., "sipping coffee near the Eiffel Tower").

    Returns:
        Success example:
        {
            "status": "success",
            "place": "Paris",
            "quote": "Evening drifts softly over Paris, cradling quiet moments in warm golden light."
        }

        Error example:
        {
            "status": "error",
            "error_message": "OpenAI quote generation failed: ..."
        }
    """

    # Validate input dicts
    if weather.get("status") != "success":
        return {"status": "error", "error_message": "Invalid or missing weather data"}

    if time.get("status") != "success":
        return {"status": "error", "error_message": "Invalid or missing time data"}

    # Extract fields
    condition = weather.get("condition")
    temperature = weather.get("temperature_c")
    weather_mood = weather.get("condition")  # you may add mood mapping later

    time_of_day = time.get("time_of_day")
    time_mood = time.get("time_mood")
    local_time = time.get("local_datetime")

    # Prepare prompt
    prompt = f"""
Generate a short, poetic quote (max 20 words) suitable for a minimalist postcard.

Context:
- Place: {place}
- Weather: {condition}, {temperature}°C
- Weather mood: {weather_mood}
- Time: {local_time} ({time_of_day})
- Time mood: {time_mood}

User context: "{user_context}"

Tone:
- Soft, emotional, atmospheric
- Minimalist wording
- Avoid clichés
- No emojis
- Should feel like a single captured moment

Output ONLY the quote text, no explanations.
"""

    try:
        response = client.responses.create(
            model="gpt-4.1-mini",
            input=prompt,
            temperature=0.7,
            max_output_tokens=50,
        )

        quote = response.output_text.strip()

        return {
            "status": "success",
            "place": place,
            "quote": quote
        }

    except Exception as e:
        return {"status": "error", "error_message": f"OpenAI quote generation failed: {str(e)}"}


**Build a tool to generate the glyph like design that resembles a city's
skyline, landscape, density, rhythm, and emotional atmosphere**

In [ ]:
def city_pulse_signature(place: str) -> dict:
    """
    Generates a one-line city pulse signature that expresses the essence of any city.
    """

    prompt = f"""
You generate a single-line “city pulse signature”: a text waveform that expresses a city's
skyline, landscape, density, rhythm, and emotional atmosphere.

This system must work for ANY city in the world with no special-case rules.

Infer from widely known attributes:
- skyline height (tall, medium, flat)
- landscape type (coastal, river, mountain, desert, plains, forest)
- cultural rhythm (calm, lively, structured, chaotic, historic, modern)
- density (open vs compact)
- emotional vibe (soft, sharp, flowing, vertical, horizontal, energetic)

Translate these qualities into symbols using ONLY:

HEIGHT BLOCKS (verticality):
▁ ▂ ▃ ▄ ▅ ▆ ▇ █

WAVES (fluidity: water, coasts, rivers, rolling hills):
~ ∿ ≋ ≈

SPIKES (peaks, sharp energy, rugged shapes):
/ \\ | ^

RHYTHMIC MARKS (tempo + cultural vibe):
· * ✦ ✧

DENSITY BLOCKS (mass, compactness, urban weight):
░ ▒ ▓

Universal aesthetic rules:
- Choose exactly 2–3 symbol families that best match the city.
- The waveform must feel coherent, intentional, and consistent.
- Length: 20–50 characters.
- NO emojis, hearts, pictograms, ASCII drawings, or extra characters.
- Do NOT attempt to draw recognizable shapes or landmarks.
- The line should feel like the city’s essence, not a picture.

City: {place}

Output only the single pulse line.
"""

    try:
        response = client.responses.create(
            model="gpt-4.1-mini",
            input=prompt,
            max_output_tokens=80,
            temperature=0.8
        )

        pulse = response.output_text.strip()

        # safety filter (ensures no rogue characters)
        allowed = set("▁▂▃▄▅▆▇█~∿≋≈/\\|^·*✦✧░▒▓ ")
        cleaned = "".join(c for c in pulse if c in allowed)

        return {
            "status": "success",
            "place": place,
            "pulse": cleaned
        }

    except Exception as e:
        return {
            "status": "error",
            "error_message": f"Pulse generation failed: {str(e)}"
        }

**Create a tool that takes information from all other tools and gives a summary**

In [ ]:
def summarize_place_state(
    place: str,
    weather: dict,
    time: dict,
    quote: dict,
    pulse: dict,
    user_context: str = None
) -> dict:
    """
    Combines weather, time-of-day, pulse signature, and quote into
    one aesthetic summary object.

    Returns:
        {
            "status": "success",
            "place": "Paris",
            "summary": "...",
            "blocks": {
                "weather": "...",
                "time": "...",
                "pulse": "...",
                "quote": "..."
            }
        }
    """

    # Handle failures gracefully
    if weather.get("status") != "success":
        return {"status": "error", "error_message": "Weather lookup failed"}
    if time.get("status") != "success":
        return {"status": "error", "error_message": "Time lookup failed"}
    if pulse.get("status") != "success":
        return {"status": "error", "error_message": "Pulse signature failed"}
    if quote.get("status") != "success":
        return {"status": "error", "error_message": "Quote generation failed"}

    weather_block = f"{weather['glyph']}  {weather['condition'].title()}  ({weather['temperature_c']}°C)"
    time_block = f"{time['glyph']}  {time['time_of_day'].title()}"
    pulse_block = pulse["pulse"]
    quote_block = quote["quote"]

    # Optional context injection
    context_line = f"\nContext: {user_context}" if user_context else ""

    # Combine into final beautiful layout
    summary = f"""
Place: **{place.title()}**

**Weather**
{weather_block}

**Time**
{time_block}

**City Pulse**

`{pulse_block}`

**Atmospheric Quote**
{quote_block}
    """.strip()

    return {
        "summary": summary
    }


**Create a tool that takes the summary and generates a html file based on the user's desired font**

In [ ]:
def build_summary_html(summary: str, font: str = None) -> str:
    """
    Generates a beautiful HTML page for the place summary.

    Args:
        summary: The multiline summary text from summarize_place_state().
        font: Optional Google Font name. If None, defaults to 'Tangerine'.

    Returns:
        A full HTML string.
    """

    # Default font
    if font is None:
        font = "Tangerine"

    # Convert summary newlines → <br> for HTML display
    formatted = summary.replace("\n", "<br>")

    # Construct HTML
    html = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <link href="https://fonts.googleapis.com/css2?family={font.replace(" ", "+")}" rel="stylesheet">
    <style>
        body {{
            font-family: '{font}', serif;
            font-size: 32px;
            background: #faf7f2;
            padding: 40px;
            line-height: 1.5;
        }}
        .container {{
            max-width: 900px;
            margin: auto;
            padding: 40px;
            border-radius: 12px;
            background: white;
            box-shadow: 0 6px 20px rgba(0,0,0,0.08);
        }}
        h1 {{
            font-size: 52px;
            margin-bottom: 20px;
        }}
        .section {{
            margin-bottom: 25px;
        }}
    </style>
</head>

<body>
    <div class="container">
        <div class="section">{formatted}</div>
    </div>
</body>
</html>
    """.strip()

    # Save file
    output_dir = "outputs"
    Path(output_dir).mkdir(exist_ok=True)

    # File path for HF Spaces
    output_path = f"{output_dir}/summary_postcard.html"

    Path(output_path).write_text(html, encoding="utf-8")

    return output_path

**Create helper functions to simplify the tools consumption in gradio**

In [ ]:
def generate_place_quote_simple(place, user_context=""):
    weather = get_weather_by_place(place)
    time = get_time_by_place(place)
    return generate_place_quote(place, weather, time, user_context)


def summarize_place_state_simple(place: str, user_context: str = ""):
    """Simplified version: only requires place + optional context."""

    # Auto-fetch everything
    weather = get_weather_by_place(place)
    time = get_time_by_place(place)
    pulse = city_pulse_signature(place)
    quote = generate_place_quote(place, weather, time, user_context)

    # Summarize
    result = summarize_place_state(
        place,
        weather,
        time,
        quote,
        pulse,
        user_context=user_context
    )

    # If error — return a clean message for Gradio
    if result.get("status") == "error":
        return f"Error: {result['error_message']}"

    return result["summary"]

**Create interface for each tool**

In [ ]:
weather_interface = gr.Interface(
    fn=get_weather_by_place,
    inputs=gr.Textbox(
        label="Enter a place",
        placeholder="New york, Tokyo",
        lines=5
    ),
    outputs=gr.Textbox(label="get weather details", lines=10),
    api_name="get_weather_by_place"
)
time_interface = gr.Interface(
    fn=get_time_by_place,
    inputs=gr.Textbox(
        label="Enter a place",
        placeholder="New york, Tokyo",
        lines=5
    ),
    outputs=gr.Textbox(label="get current time details", lines=10),
    api_name="get_time_by_place"
)
quote_interface = gr.Interface(
    fn=generate_place_quote_simple,
    inputs=[
        gr.Textbox(label="Place"),
        gr.Textbox(label="User Context")
    ],
    outputs=gr.Textbox(label="Minimalistic Quote", lines=5),
    api_name="generate_place_quote"
)
city_pulse_interface = gr.Interface(
    fn=city_pulse_signature,
    inputs=gr.Textbox(
        label="Enter a place",
        placeholder="New york, Tokyo",
        lines=5
    ),
    outputs=gr.Textbox(label="get city pulse signature", lines=10),
    api_name="city_pulse_signature"
)
summary_interface = gr.Interface(
    fn=summarize_place_state_simple,
    inputs=[
        gr.Textbox(label="Place"),
        gr.Textbox(label="What are you doing? (optional)")
    ],
    outputs=gr.Markdown(label="Minimalistic Postcard Summary"),
    api_name="summarize_place_simple"
)
html_interface = gr.Interface(
    fn=build_summary_html,
    inputs=[
        gr.Textbox(label="Markdown Input", lines=12, placeholder="# Your Markdown here..."),
        gr.Textbox(label="Font (optional)", placeholder="Tangerine (default)")
    ],
    outputs=gr.File(label="Download HTML"),
    title="Markdown → Styled HTML Converter",
    description="Converts markdown into a clean HTML file styled with Google Fonts."
)

**Create gradio UI with mcp_server = True to convert to mcp server**

In [ ]:
with gr.Blocks(title="A soft atmospheric postcard generated from the city’s moment.") as demo:
    gr.Markdown("""
        ## Minimalistic Postcard: An MCP server that gives a calm, stripped-down postcard of place and feeling.
    """)

    gr.TabbedInterface(
        [
            weather_interface,
            time_interface,
            quote_interface,
            city_pulse_interface,
            summary_interface,
            html_interface

        ],
        [
            "weather_interface",
            "time_interface",
            "quote_interface",
            "city_pulse_interface",
            "summary_interface",
            "html_interface"

        ]

    )
    gr.Markdown("""
# 🎥 [Watch the Demo](https://youtu.be/-gaDxKcdHTA?si=X5H8wtBvQwQpQ1fv)
""")
    gr.Markdown("""
## **Example prompt to use on claude**: I am in tokyo, eating sushi near tokyo tower, generate a beautiful minimalistic postcard with tangerine font html file
""")


if __name__ == "__main__":
    demo.launch(theme = gr.themes.Monochrome(),mcp_server = True)